In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-question-pairs/train.csv.zip
/kaggle/input/quora-question-pairs/sample_submission.csv.zip
/kaggle/input/quora-question-pairs/test.csv
/kaggle/input/quora-question-pairs/test.csv.zip


In [2]:
!pip -q install sentence-transformers

import zipfile
import pandas as pd
from sklearn import model_selection
from datasets import Dataset

from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.similarity_functions import SimilarityFunction
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.training_args import SentenceTransformerTrainingArguments


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 17.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.6 MB/s eta 0:00:0000:0100:01


2025-09-07 12:58:26.645186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757249906.979224      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757249907.077764      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Unzip train.csv.zip
with zipfile.ZipFile("/kaggle/input/quora-question-pairs/train.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("./train/")

# Read CSV
df = pd.read_csv("./train/train.csv")

# Drop missing values
df = df.dropna()

# Rename for clarity
df = df.rename(columns={'is_duplicate': 'label'})
df = df[['question1', 'question2', 'label']]

print(df.shape)
df.head()


(404287, 3)


,question1,question2,label
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
train_df, val_df = model_selection.train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

# Convert to HuggingFace datasets
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds = Dataset.from_pandas(val_df.reset_index(drop=True))


In [5]:
config = {
    "model_path": "microsoft/xtremedistil-l6-h256-uncased",  # Default model
    "learning_rate": 5e-4,
    "train_batch_size": 64,  # Keep this lower if you get CUDA OOM errors
    "eval_batch_size": 64,
    "epochs": 2,  # Start with 2 for quick test, increase later
    "warmup_ratio": 0.1,
    "output_dir": "./model_output"
}


In [6]:
# Load Sentence Transformer
model = SentenceTransformer(config["model_path"])

# Choose loss function (CosineSimilarityLoss here)
train_loss = losses.CosineSimilarityLoss(model=model)


config.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/51.0M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/51.0M [00:00<?, ?B/s]

In [7]:
# Create evaluator
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_ds["question1"],
    sentences2=val_ds["question2"],
    scores=val_ds["label"],
    main_similarity=SimilarityFunction.COSINE,
    name="dev-score"
)

# Evaluate before training
print("Initial Evaluation:")
dev_evaluator(model)


Initial Evaluation:


{'dev-score_pearson_cosine': 0.2790310974139594,
 'dev-score_spearman_cosine': 0.3328206741129431}

In [9]:
args = SentenceTransformerTrainingArguments(
    output_dir=config["output_dir"],
    num_train_epochs=config["epochs"],
    learning_rate=config["learning_rate"],
    per_device_train_batch_size=config["train_batch_size"],
    per_device_eval_batch_size=config["eval_batch_size"],
    warmup_ratio=config["warmup_ratio"],
    fp16=True,
    save_total_limit=1,
    report_to="none"
)


In [10]:
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    loss=train_loss,
    evaluator=dev_evaluator
)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [11]:
trainer.train()


Step,Training Loss
500,0.216800
1000,0.203000
1500,0.204300
2000,0.197300
2500,0.200500
3000,0.203500
3500,0.193200
4000,0.184700
4500,0.179500
5000,0.173100


TrainOutput(global_step=5054, training_loss=0.19529541556807814, metrics={'train_runtime': 641.3261, 'train_samples_per_second': 1008.626, 'train_steps_per_second': 7.881, 'total_flos': 0.0, 'train_loss': 0.19529541556807814, 'epoch': 2.0})

In [12]:
model.save(config["output_dir"])


In [13]:
print("Final Evaluation on Validation Set:")
dev_evaluator(model)


Final Evaluation on Validation Set:


{'dev-score_pearson_cosine': 0.43464684541836407,
 'dev-score_spearman_cosine': 0.459682264345699}

contrastive loss

In [14]:
from sentence_transformers import losses

# Create a new model from the same base
model_contrastive = SentenceTransformer(config["model_path"])

# Contrastive Loss
train_loss_contrastive = losses.ContrastiveLoss(model=model_contrastive)

# Trainer
trainer_contrastive = SentenceTransformerTrainer(
    model=model_contrastive,
    args=args,  # same training args as before
    train_dataset=train_ds,
    eval_dataset=val_ds,
    loss=train_loss_contrastive,
    evaluator=dev_evaluator
)

# Train
trainer_contrastive.train()

# Save model
model_contrastive.save(config["output_dir"] + "/contrastive_model")

# Final Evaluation
print("Final Evaluation with Contrastive Loss:")
dev_evaluator(model_contrastive)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.024500
1000,0.022000
1500,0.020100
2000,0.019100
2500,0.017900
3000,0.016400
3500,0.015500
4000,0.015000
4500,0.014500
5000,0.014300


Final Evaluation with Contrastive Loss:


{'dev-score_pearson_cosine': 0.6566195014207411,
 'dev-score_spearman_cosine': 0.7010676186402595}

negative ranking loss

In [15]:
from sentence_transformers import losses

# Create a new model from the same base
model_mnr = SentenceTransformer(config["model_path"])

# Multiple Negative Ranking Loss
train_loss_mnr = losses.MultipleNegativesRankingLoss(model=model_mnr)

# Trainer
trainer_mnr = SentenceTransformerTrainer(
    model=model_mnr,
    args=args,  # reuse the same args
    train_dataset=train_ds,
    eval_dataset=val_ds,
    loss=train_loss_mnr,
    evaluator=dev_evaluator
)

# Train
trainer_mnr.train()

# Save model
model_mnr.save(config["output_dir"] + "/mnr_model")

# Final Evaluation
print("Final Evaluation with Multiple Negatives Ranking Loss:")
dev_evaluator(model_mnr)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,1.550600
1000,1.042800
1500,0.921700
2000,0.869700
2500,0.818200
3000,0.704100
3500,0.672200
4000,0.652400
4500,0.636900
5000,0.632600


Final Evaluation with Multiple Negatives Ranking Loss:


{'dev-score_pearson_cosine': 0.4709590285951696,
 'dev-score_spearman_cosine': 0.5109328337860535}

cross-encoder 

In [1]:
from torch.utils.data import DataLoader
from sentence_transformers import CrossEncoder, InputExample
from sklearn.metrics import f1_score

# Prepare CrossEncoder training data
train_samples = [
    InputExample(texts=[q1, q2], label=float(label))
    for q1, q2, label in zip(train_ds["question1"], train_ds["question2"], train_ds["label"])
]
train_samples = train_samples[:20000]  # Only first 20k pairs

val_samples = [
    InputExample(texts=[q1, q2], label=float(label))
    for q1, q2, label in zip(val_ds["question1"], val_ds["question2"], val_ds["label"])
]

# Create CrossEncoder model
cross_encoder = CrossEncoder("microsoft/xtremedistil-l6-h256-uncased", num_labels=1)

# Create DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=32)

# Train CrossEncoder
cross_encoder.fit(
    train_dataloader=train_dataloader,
    epochs=1,  # Increase for better performance
    warmup_steps=100,
    output_path="./cross_encoder_model"
)

# Evaluation on validation set
preds = cross_encoder.predict([[ex.texts[0], ex.texts[1]] for ex in val_samples])
preds_binary = [1 if p > 0.5 else 0 for p in preds]
val_labels = [int(ex.label) for ex in val_samples]

f1 = f1_score(val_labels, preds_binary)
print(f"Final F1-Score for Cross-Encoder: {f1:.4f}")

2025-09-07 19:18:58.085662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757272738.413916      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757272738.509252      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


NameError: name 'train_ds' is not defined